In [1]:
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from bokeh.layouts import gridplot

In [2]:
output_notebook()

Loading BokehJS ...

### Notes:
- We should plot 'Interquartile Range'
- Plot as violine or box plot. Organism across frag sizes or frag sizes across oragnisms.

In [3]:
def get_genome_sequence(path_to_fasta):
    genome_sequence_lines = []
    with open(path_to_fasta, 'r') as genome_file:
        next(genome_file)
        for line in genome_file:
            genome_sequence_lines.append(line.rstrip())  

    return ''.join(genome_sequence_lines)

In [4]:
def fractionate_sequence(sequence, fraction_length):
    return (sequence[0+i:fraction_length+i] for i in range(0, len(sequence), fraction_length))

In [5]:
 def calculate_gc_content(sequence): 
    nucleotide_counts = dict((char.upper(), sequence.count(char)) for char in set(sequence))

    total_nucleotides = 0
    total_gc = 0

    for base, count in nucleotide_counts.items():
        if base == 'G' or base == 'C' or base == 'S': # 'S' IUPAC code for G or C
            total_gc = total_gc + count
            total_nucleotides = total_nucleotides + count
        elif base == 'A' or base == 'T' or base == 'W': # 'W' IUPAC code for A or T
            total_nucleotides = total_nucleotides + count
        else:
            continue # Skip N's or other ambigous bases.

    gc_content = (total_gc / total_nucleotides) * 100
    
    return gc_content

In [6]:
def create_gc_fractionation_plot(gc_contents, current_sheer_size, global_gc_content, organism_name): 
    title_text = 'Variation in fragment GC content after simulated sheering of the {} genome to {} kbp fragments.'.format(organism_name, int(current_sheer_size / 1000))
    
    fraction_count = len(gc_contents)
    base_figure = figure(plot_width=800, 
                         plot_height=600, 
                         y_range=(20,80), 
                         x_range=(0,fraction_count), 
                         title=title_text,
                         output_backend="webgl")
    
    # change just some things about the x-grid
    base_figure.xgrid.grid_line_color = None

    # change just some things about the y-grid
    base_figure.ygrid.grid_line_alpha = 0.5
    base_figure.ygrid.grid_line_dash = [6, 4]
    final_figure = add_factionated_gc_content_for_organism(base_figure, gc_contents, global_gc_content, 'firebrick', 'black', organism_name)
    
    return final_figure

In [7]:
def add_factionated_gc_content_for_organism(figure, gc_contents, organism_global_gc_content, 
                                            local_color, global_colour, organism_name):
    fraction_count = len(gc_contents)

    figure.xaxis.axis_label = "Position of fragment on {} genome.".format(organism_name)
    figure.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
    figure.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks
    figure.xaxis.major_label_text_font_size = '0pt'  # turn off x-axis tick labels
    figure.yaxis.axis_label = "Percent GC Content"

    figure.line(range(0, len(gc_contents)), gc_contents, line_width=0.5, color=local_color)

    figure.hbar(y=global_gc_content, height=0.2, left=0,
           right=fraction_count, color=global_colour, legend='Mean GC content of the {} genome.'.format(organism_name))
    
    return figure

In [8]:
ecoli_sequence = get_genome_sequence('./testing/test_constants/NC_002695.1.fasta')

In [9]:
global_gc_content = calculate_gc_content(ecoli_sequence)

In [10]:
sheer_sizes = [1000, 2000, 5000, 10000, 20000, 50000, 100000, 500000]

In [11]:
figure_list = []
for size in sheer_sizes:
    sequence_fractions = fractionate_sequence(ecoli_sequence, size)
    gc_contents = list(map(calculate_gc_content,sequence_fractions))
    new_figure = create_gc_fractionation_plot(gc_contents, size, global_gc_content, 'Escherichia coli O157:H7')
    figure_list.append(new_figure)

In [12]:
grid = gridplot(figure_list, ncols=2)

In [13]:
output_file("ecoli_fraction_viz.html", title="Escherichia coli O157:H7 GC Content Fractionation Viz.")

In [14]:
show(grid)